## Logit lens

Trying logit lens

In [8]:
import numpy as numpy
import plotly.express as px
import torch

import circuitsvis as cv

import transformer_lens.utils as utils
from transformer_lens.hook_points import HookPoint
from transformer_lens import HookedTransformer, FactoredMatrix
import einops

import tqdm.auto as tqdm
from functools import partial

In [3]:
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

In [4]:
def vis_attn_patterns(model, text, layers, compact=True):
    str_tokens = model.to_str_tokens(text)
    logits, cache = model.run_with_cache(text, remove_batch_dim=True)

    if compact:
        for layer in layers:
            attention_pattern = cache["pattern", layer]
            display(cv.attention.attention_patterns(tokens=str_tokens, attention=attention_pattern))
    
    else:
        for layer in layers:
            attention_pattern = cache["pattern", layer]
            display(cv.attention.attention_heads(tokens=str_tokens, attention=attention_pattern))

In [5]:
model = HookedTransformer.from_pretrained("gpt2-small")

Loaded pretrained model gpt2-small into HookedTransformer


In [6]:
text = "The capital of Norway is"
answer = " Oslo"

logits, cache = model.run_with_cache(text)

In [16]:
acc_resid_stack = cache.accumulated_resid(pos_slice=-1)
scaled_resid_stack = cache.apply_ln_to_stack(acc_resid_stack, pos_slice=-1)
print(scaled_resid_stack.shape)
Unembedding_matrix = model.W_U
print(Unembedding_matrix.shape)
#n_layers, _, n_dim = scaled_resid_stack.shape

logit_lens_final_logits = einops.einsum(scaled_resid_stack, Unembedding_matrix,
                                        "n_layer ... n_dim, n_dim vocab -> n_layer vocab")
print(logit_lens_final_logits.shape)

torch.Size([13, 1, 768])
torch.Size([768, 50257])
torch.Size([13, 50257])


In [18]:
from "Decoder_functions.py" import get_predictions

for layer in range(len(logit_lens_final_logits)):
    print(model.to_string(logit_lens_final_logits[layer]))

SyntaxError: invalid syntax (2264332087.py, line 1)